In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc, f1_score
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
import random

random.seed(0)
weight_decay = 0.0005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'fashion-mnist': ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag',
                          'ankle-boot'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print('training class:',single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]
    
    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    print('Start of the proposed transformation')
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print('End of the proposed transformation')
    batch_size = 128
    print('Start of the training')
    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=batch_size, epochs= 5 #int(np.ceil(200/transformer.n_transforms)) #N_EPOCHS
            )
    print('End of the training')

    #Estimating the parameters 
    def calc_approx_alpha_sum(observations):
        N = len(observations)
        f = np.mean(observations, axis=0)

        return (N * (len(f) - 1) * (-psi(1))) / (
                N * np.sum(f * np.log(f)) - np.sum(f * np.sum(np.log(observations), axis=0)))

    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x

    def fixed_point_dirichlet_mle(alpha_init, log_p_hat, max_iter=10):
        alpha_new = alpha_old = alpha_init
        for _ in range(max_iter):
            alpha_new = inv_psi(np.clip(psi(np.clip(np.sum(alpha_old),10**(-10),None)) + log_p_hat,None,10))
            if np.sqrt(np.sum((alpha_old - alpha_new) ** 2)) < 1e-5:
                break
            alpha_old = alpha_new
        return alpha_new

    #Normality Score
    def dirichlet_normality_score(alpha, p):
        return np.sum((alpha - 1) * np.log(p), axis=-1)

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=64)
        log_p_hat_train = np.log(np.clip(observed_dirichlet,10**(-10),1)).mean(axis=0)

        alpha_sum_approx = calc_approx_alpha_sum(observed_dirichlet)
        alpha_sum_approx = np.clip(alpha_sum_approx, None,10**7)
        alpha_0 = observed_dirichlet.mean(axis=0) * alpha_sum_approx
        alpha_0 = np.nan_to_num(alpha_0, copy=True, nan=10**3, posinf=10**5, neginf=-(10**5))

        mle_alpha_t = fixed_point_dirichlet_mle(alpha_0, log_p_hat_train)
        mle_alpha_t = np.nan_to_num(mle_alpha_t, copy=True, nan=10**3, posinf=10**5, neginf=-(10**5))

        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=64)
        scores += dirichlet_normality_score(mle_alpha_t, x_test_p)
        scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**10, neginf=-(10**10))

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    # Transformations
    for i in range(n_runs):
        print('run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]

        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        (load_fashion_mnist, 'fashion-mnist', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
run number: 1
training class: 0
Wide Residual Network-16-8 created.
Start of the proposed transformation
End of the proposed transformation
Start of the training
Epoch 1/5


204/530 [==========>...................] - ETA: 29:25 - loss: 8.8368 - acc: 0.00 - ETA: 19:32 - loss: 8.7415 - acc: 0.00 - ETA: 18:25 - loss: 8.6445 - acc: 0.02 - ETA: 18:08 - loss: 8.5483 - acc: 0.03 - ETA: 17:47 - loss: 8.4536 - acc: 0.04 - ETA: 17:37 - loss: 8.3609 - acc: 0.04 - ETA: 17:42 - loss: 8.2708 - acc: 0.05 - ETA: 17:33 - loss: 8.1821 - acc: 0.05 - ETA: 17:27 - loss: 8.0961 - acc: 0.06 - ETA: 17:21 - loss: 8.0116 - acc: 0.06 - ETA: 17:16 - loss: 7.9297 - acc: 0.07 - ETA: 17:10 - loss: 7.8494 - acc: 0.07 - ETA: 17:06 - loss: 7.7710 - acc: 0.07 - ETA: 17:09 - loss: 7.6949 - acc: 0.07 - ETA: 17:05 - loss: 7.6209 - acc: 0.07 - ETA: 17:00 - loss: 7.5496 - acc: 0.07 - ETA: 16:57 - loss: 7.4797 - acc: 0.07 - ETA: 16:59 - loss: 7.4113 - acc: 0.07 - ETA: 17:00 - loss: 7.3455 - acc: 0.07 - ETA: 17:00 - loss: 7.2812 - acc: 0.08 - ETA: 16:57 - loss: 7.2177 - acc: 0.08 - ETA: 16:54 - loss: 7.1571 - acc: 0.08 - ETA: 16:50 - loss: 7.0981 - acc: 0.08 - ETA: 16:49 - loss: 7.0406 - acc: 0.09

408/530 [======================>.......] - ETA: 10:30 - loss: 2.9794 - acc: 0.37 - ETA: 10:28 - loss: 2.9716 - acc: 0.37 - ETA: 10:26 - loss: 2.9639 - acc: 0.37 - ETA: 10:24 - loss: 2.9563 - acc: 0.37 - ETA: 10:22 - loss: 2.9487 - acc: 0.37 - ETA: 10:20 - loss: 2.9412 - acc: 0.37 - ETA: 10:18 - loss: 2.9337 - acc: 0.37 - ETA: 10:16 - loss: 2.9264 - acc: 0.37 - ETA: 10:14 - loss: 2.9193 - acc: 0.37 - ETA: 10:12 - loss: 2.9122 - acc: 0.37 - ETA: 10:10 - loss: 2.9049 - acc: 0.37 - ETA: 10:08 - loss: 2.8977 - acc: 0.37 - ETA: 10:06 - loss: 2.8906 - acc: 0.37 - ETA: 10:04 - loss: 2.8835 - acc: 0.37 - ETA: 10:02 - loss: 2.8763 - acc: 0.37 - ETA: 10:00 - loss: 2.8694 - acc: 0.38 - ETA: 9:58 - loss: 2.8625 - acc: 0.3809 - ETA: 9:56 - loss: 2.8557 - acc: 0.381 - ETA: 9:54 - loss: 2.8490 - acc: 0.381 - ETA: 9:52 - loss: 2.8424 - acc: 0.381 - ETA: 9:50 - loss: 2.8358 - acc: 0.382 - ETA: 9:48 - loss: 2.8293 - acc: 0.382 - ETA: 9:46 - loss: 2.8227 - acc: 0.383 - ETA: 9:45 - loss: 2.8163 - acc: 0.38

530/530 [==============================] - ETA: 3:54 - loss: 2.0997 - acc: 0.479 - ETA: 3:52 - loss: 2.0973 - acc: 0.479 - ETA: 3:50 - loss: 2.0948 - acc: 0.480 - ETA: 3:48 - loss: 2.0926 - acc: 0.480 - ETA: 3:46 - loss: 2.0901 - acc: 0.481 - ETA: 3:44 - loss: 2.0879 - acc: 0.481 - ETA: 3:42 - loss: 2.0854 - acc: 0.481 - ETA: 3:40 - loss: 2.0831 - acc: 0.482 - ETA: 3:38 - loss: 2.0805 - acc: 0.483 - ETA: 3:36 - loss: 2.0782 - acc: 0.483 - ETA: 3:34 - loss: 2.0759 - acc: 0.484 - ETA: 3:33 - loss: 2.0734 - acc: 0.484 - ETA: 3:31 - loss: 2.0711 - acc: 0.484 - ETA: 3:29 - loss: 2.0688 - acc: 0.485 - ETA: 3:27 - loss: 2.0667 - acc: 0.485 - ETA: 3:25 - loss: 2.0646 - acc: 0.485 - ETA: 3:23 - loss: 2.0620 - acc: 0.486 - ETA: 3:21 - loss: 2.0597 - acc: 0.486 - ETA: 3:19 - loss: 2.0573 - acc: 0.487 - ETA: 3:17 - loss: 2.0548 - acc: 0.487 - ETA: 3:15 - loss: 2.0526 - acc: 0.488 - ETA: 3:13 - loss: 2.0503 - acc: 0.488 - ETA: 3:11 - loss: 2.0479 - acc: 0.489 - ETA: 3:09 - loss: 2.0457 - acc: 0.489

204/530 [==========>...................] - ETA: 16:48 - loss: 1.0198 - acc: 0.71 - ETA: 16:52 - loss: 0.9940 - acc: 0.71 - ETA: 16:49 - loss: 0.9794 - acc: 0.72 - ETA: 17:04 - loss: 0.9851 - acc: 0.72 - ETA: 16:59 - loss: 0.9949 - acc: 0.72 - ETA: 16:56 - loss: 0.9853 - acc: 0.73 - ETA: 17:09 - loss: 0.9863 - acc: 0.72 - ETA: 17:04 - loss: 0.9786 - acc: 0.73 - ETA: 17:01 - loss: 0.9797 - acc: 0.72 - ETA: 16:57 - loss: 0.9778 - acc: 0.72 - ETA: 16:52 - loss: 0.9803 - acc: 0.71 - ETA: 16:49 - loss: 0.9770 - acc: 0.72 - ETA: 16:46 - loss: 0.9699 - acc: 0.72 - ETA: 16:45 - loss: 0.9694 - acc: 0.72 - ETA: 16:42 - loss: 0.9771 - acc: 0.72 - ETA: 16:39 - loss: 0.9805 - acc: 0.71 - ETA: 16:36 - loss: 0.9753 - acc: 0.71 - ETA: 16:33 - loss: 0.9722 - acc: 0.72 - ETA: 16:30 - loss: 0.9669 - acc: 0.72 - ETA: 16:28 - loss: 0.9684 - acc: 0.72 - ETA: 16:26 - loss: 0.9686 - acc: 0.72 - ETA: 16:23 - loss: 0.9677 - acc: 0.72 - ETA: 16:21 - loss: 0.9654 - acc: 0.72 - ETA: 16:20 - loss: 0.9632 - acc: 0.72

...

408/563 [====================>.........] - ETA: 11:27 - loss: 0.1606 - acc: 0.99 - ETA: 11:25 - loss: 0.1605 - acc: 0.99 - ETA: 11:23 - loss: 0.1605 - acc: 0.99 - ETA: 11:22 - loss: 0.1604 - acc: 0.99 - ETA: 11:20 - loss: 0.1604 - acc: 0.99 - ETA: 11:18 - loss: 0.1604 - acc: 0.99 - ETA: 11:16 - loss: 0.1603 - acc: 0.99 - ETA: 11:14 - loss: 0.1602 - acc: 0.99 - ETA: 11:12 - loss: 0.1602 - acc: 0.99 - ETA: 11:10 - loss: 0.1601 - acc: 0.99 - ETA: 11:08 - loss: 0.1600 - acc: 0.99 - ETA: 11:06 - loss: 0.1600 - acc: 0.99 - ETA: 11:04 - loss: 0.1599 - acc: 0.99 - ETA: 11:03 - loss: 0.1599 - acc: 0.99 - ETA: 11:01 - loss: 0.1598 - acc: 0.99 - ETA: 10:59 - loss: 0.1598 - acc: 0.99 - ETA: 10:57 - loss: 0.1597 - acc: 0.99 - ETA: 10:55 - loss: 0.1597 - acc: 0.99 - ETA: 10:53 - loss: 0.1596 - acc: 0.99 - ETA: 10:51 - loss: 0.1596 - acc: 0.99 - ETA: 10:49 - loss: 0.1595 - acc: 0.99 - ETA: 10:47 - loss: 0.1594 - acc: 0.99 - ETA: 10:45 - loss: 0.1594 - acc: 0.99 - ETA: 10:44 - loss: 0.1593 - acc: 0.99

563/563 [==============================] - ETA: 4:56 - loss: 0.1456 - acc: 0.998 - ETA: 4:54 - loss: 0.1456 - acc: 0.998 - ETA: 4:52 - loss: 0.1457 - acc: 0.998 - ETA: 4:50 - loss: 0.1458 - acc: 0.998 - ETA: 4:48 - loss: 0.1459 - acc: 0.998 - ETA: 4:46 - loss: 0.1461 - acc: 0.998 - ETA: 4:44 - loss: 0.1461 - acc: 0.998 - ETA: 4:42 - loss: 0.1464 - acc: 0.998 - ETA: 4:40 - loss: 0.1465 - acc: 0.998 - ETA: 4:38 - loss: 0.1467 - acc: 0.998 - ETA: 4:36 - loss: 0.1467 - acc: 0.998 - ETA: 4:34 - loss: 0.1468 - acc: 0.998 - ETA: 4:33 - loss: 0.1470 - acc: 0.998 - ETA: 4:31 - loss: 0.1471 - acc: 0.998 - ETA: 4:29 - loss: 0.1473 - acc: 0.998 - ETA: 4:27 - loss: 0.1473 - acc: 0.998 - ETA: 4:25 - loss: 0.1474 - acc: 0.998 - ETA: 4:23 - loss: 0.1474 - acc: 0.998 - ETA: 4:21 - loss: 0.1477 - acc: 0.998 - ETA: 4:19 - loss: 0.1477 - acc: 0.998 - ETA: 4:17 - loss: 0.1479 - acc: 0.998 - ETA: 4:15 - loss: 0.1479 - acc: 0.998 - ETA: 4:13 - loss: 0.1481 - acc: 0.998 - ETA: 4:11 - loss: 0.1481 - acc: 0.998

204/563 [=========>....................] - ETA: 17:49 - loss: 0.1322 - acc: 1.00 - ETA: 17:51 - loss: 0.1538 - acc: 0.98 - ETA: 17:59 - loss: 0.1518 - acc: 0.99 - ETA: 17:55 - loss: 0.1589 - acc: 0.99 - ETA: 17:58 - loss: 0.1597 - acc: 0.99 - ETA: 17:57 - loss: 0.1590 - acc: 0.99 - ETA: 17:54 - loss: 0.1649 - acc: 0.99 - ETA: 17:51 - loss: 0.1656 - acc: 0.99 - ETA: 17:46 - loss: 0.1647 - acc: 0.99 - ETA: 17:44 - loss: 0.1643 - acc: 0.99 - ETA: 17:41 - loss: 0.1640 - acc: 0.99 - ETA: 17:39 - loss: 0.1675 - acc: 0.99 - ETA: 17:36 - loss: 0.1663 - acc: 0.99 - ETA: 17:36 - loss: 0.1659 - acc: 0.99 - ETA: 17:34 - loss: 0.1662 - acc: 0.99 - ETA: 17:31 - loss: 0.1660 - acc: 0.99 - ETA: 17:29 - loss: 0.1647 - acc: 0.99 - ETA: 17:26 - loss: 0.1640 - acc: 0.99 - ETA: 17:25 - loss: 0.1630 - acc: 0.99 - ETA: 17:24 - loss: 0.1627 - acc: 0.99 - ETA: 17:22 - loss: 0.1619 - acc: 0.99 - ETA: 17:21 - loss: 0.1628 - acc: 0.99 - ETA: 17:19 - loss: 0.1620 - acc: 0.99 - ETA: 17:17 - loss: 0.1629 - acc: 0.99

408/563 [====================>.........] - ETA: 11:27 - loss: 0.1653 - acc: 0.99 - ETA: 11:25 - loss: 0.1652 - acc: 0.99 - ETA: 11:23 - loss: 0.1651 - acc: 0.99 - ETA: 11:21 - loss: 0.1653 - acc: 0.99 - ETA: 11:19 - loss: 0.1652 - acc: 0.99 - ETA: 11:18 - loss: 0.1652 - acc: 0.99 - ETA: 11:16 - loss: 0.1653 - acc: 0.99 - ETA: 11:14 - loss: 0.1653 - acc: 0.99 - ETA: 11:12 - loss: 0.1652 - acc: 0.99 - ETA: 11:10 - loss: 0.1653 - acc: 0.99 - ETA: 11:08 - loss: 0.1654 - acc: 0.99 - ETA: 11:06 - loss: 0.1654 - acc: 0.99 - ETA: 11:04 - loss: 0.1653 - acc: 0.99 - ETA: 11:02 - loss: 0.1653 - acc: 0.99 - ETA: 11:00 - loss: 0.1652 - acc: 0.99 - ETA: 10:58 - loss: 0.1652 - acc: 0.99 - ETA: 10:56 - loss: 0.1651 - acc: 0.99 - ETA: 10:54 - loss: 0.1651 - acc: 0.99 - ETA: 10:53 - loss: 0.1650 - acc: 0.99 - ETA: 10:51 - loss: 0.1649 - acc: 0.99 - ETA: 10:49 - loss: 0.1649 - acc: 0.99 - ETA: 10:47 - loss: 0.1648 - acc: 0.99 - ETA: 10:45 - loss: 0.1648 - acc: 0.99 - ETA: 10:43 - loss: 0.1647 - acc: 0.99

563/563 [==============================] - ETA: 4:55 - loss: 0.1529 - acc: 0.996 - ETA: 4:54 - loss: 0.1528 - acc: 0.996 - ETA: 4:52 - loss: 0.1527 - acc: 0.996 - ETA: 4:50 - loss: 0.1526 - acc: 0.996 - ETA: 4:48 - loss: 0.1525 - acc: 0.996 - ETA: 4:46 - loss: 0.1524 - acc: 0.996 - ETA: 4:44 - loss: 0.1523 - acc: 0.996 - ETA: 4:42 - loss: 0.1523 - acc: 0.996 - ETA: 4:40 - loss: 0.1522 - acc: 0.996 - ETA: 4:38 - loss: 0.1521 - acc: 0.996 - ETA: 4:36 - loss: 0.1520 - acc: 0.996 - ETA: 4:34 - loss: 0.1519 - acc: 0.996 - ETA: 4:32 - loss: 0.1518 - acc: 0.996 - ETA: 4:30 - loss: 0.1517 - acc: 0.996 - ETA: 4:29 - loss: 0.1517 - acc: 0.996 - ETA: 4:27 - loss: 0.1516 - acc: 0.996 - ETA: 4:25 - loss: 0.1515 - acc: 0.996 - ETA: 4:23 - loss: 0.1515 - acc: 0.996 - ETA: 4:21 - loss: 0.1514 - acc: 0.996 - ETA: 4:19 - loss: 0.1514 - acc: 0.996 - ETA: 4:17 - loss: 0.1513 - acc: 0.996 - ETA: 4:15 - loss: 0.1512 - acc: 0.996 - ETA: 4:13 - loss: 0.1513 - acc: 0.996 - ETA: 4:11 - loss: 0.1513 - acc: 0.996